<h1 align='center'> Buy low, sell high - a walkthrough </h1>

Why hello there! Ready to make a shitload of money on the stock market? Good. Here is how you do it.

## The Classes

This framework is made up of two classes of Python objects: Portfolios and Stocks. 

- A <b>Portfolio</b> object is defined by its capital and Stocks. 
  - Capital can be used to purchase shares of a Stock object and Stock shares can be sold for capital. 
  - Portfolio objects contain a ledger that records details from each transactions.
- A <b>Stock</b> object is defined by its ticker symbol. 
  - When a stock is added to a portfolio, an instance of a AlphaVantage TimeSeries object is created and both daily and intradaily data is gathered.
  - The gathered data contains the open, close, high, and low values per share and the total trade volume and is used to define the buy and sell prices for a Stock at a particular time.

## Examining the data

Before we dive into buying and selling stock, let's take a minute to understand the data the we are using. Each Stock object comes with a large amount of data that can be manipulated and aggregated at different intervals, so let's explore a little bit!

In order to start creating objects and calling functions, we need to import the classes from the python file.

In [1]:
from classes import *

Let's create a Stock object and gather it's data. Normally, this will happen behind the scenes when a Stock is added to a Portfolio, but let's go through the details here.

In [2]:
Google = Stock('GOOGL')
Google.gatherDaily()
Google.gatherIntra()

Both of the gather functions have parameters that can be adjusted. Calling the function with no parameters will use the defaults.

- <b>gatherDaily(outputsize)</b>: Returns a Pandas DataFrame with open, close, high, low, and trade volume across days
   - <b>outputsize</b> defines how much data is returned: "compact" (default) returns the last 100 datapoints, "full" returns all available data
- <b>gatherIntra(interval, outputsize)</b>: Returns a Pandas DataFrame with open, close, high, low data, and trade volume within a day at the given interval
   - <b>interval</b> defines time difference between datapoints: '1min', '5min', '15min', '30min' (default), '60min'
   - <b>outputsize</b> defines how much data is returned: "compact" (default) returns the last 100 datapoints, "full" returns all available data

These DataFrames are stored as variables on the Stock object:

In [3]:
Google.daily.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-03-13,1174.99,1214.40,1112.29,1214.27,3969973.0
2020-03-16,1089.61,1145.47,1067.08,1073.00,4825981.0
2020-03-17,1090.64,1126.00,1049.10,1118.06,4159686.0
2020-03-18,1059.94,1104.99,1037.00,1091.19,4636840.0
2020-03-19,1088.22,1152.61,1055.55,1111.67,3703248.0


In [4]:
Google.intra.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-07-28 15:30:00,1515.29,1515.7750,1506.525,1508.99,81177.0
2020-07-28 16:00:00,1508.47,1509.3200,1500.520,1504.60,208483.0
2020-07-28 16:30:00,1503.65,1510.9145,1503.650,1504.02,168435.0
2020-07-28 18:30:00,1504.01,1504.0100,1504.010,1504.01,360.0
2020-07-29 09:30:00,1503.65,1507.2900,1503.650,1507.29,13238.0


These DataFrames are used to determine the buy/sell prices at a particular time, among other things. Each Stock has functions <b>buyPrice</b> and <b>sellPrice</b> that will return the price for a given index. You can pass in an integer index (e.g., 0 for the first line in the daily dataset) or an actual date/time (e.g. '2020-07-28 15:30:00 for the first line in the intra dataset). 

The value returned is <u>the open price for the next index in the dataframe</u>.

In [5]:
Google.sellPrice(0, isDate=0, isDaily=1)

1089.61

In [6]:
Google.buyPrice('2020-07-28 15:30:00', isDate=1, isDaily=0)

1508.47

There is a similar function for trade volume:

In [7]:
Google.volume(3, isDate=0, isDaily=0)

360.0

## Creating a portfolio and buying stock

Okay, now that we have an understanding of what we're working with, let's get started. 

The first thing we will need to do is create an instance of a Portfolio. I've been watching a bunch of Archer lately, so let's set Sterling up with a new portfolio using the $100k his mommy gave him.

In [8]:
Archer = Portfolio(100000)

Now that we have a portfolio, we can buy some stock! 

Sterling would love to pour his money into his favorite things: booze and hookers. Thankfully, we were able to satisfy his urges and convince him otherwise by using Woodhouse's smack needles to inject 8 gils of liquor right into his veins and finding the cheapiest, dirtiest whore we could find off of the street. I'm talking Frank Reynolds level dirty whore. But I digress...

Archer decided to put his money into gold, water, and keep a bit left as available captial. 

Now, we can choose to buy stocks by either specifying the number of shares or the dollar amount. This is driven by the variable <b>isShares</b>, which defaults to number of shares. For now, we will split Archer's portfolio out by dollar amount, with 40k going to GOLD, 40k going to AWK, and 20k remaining as capital.

In [9]:
Archer.buy('GOLD', 40000, isShares=0)

1383.6 shares of GOLD were purchased for $40000


In [13]:
Archer.buy('AWK', 40000, isShares=0)

277.24 shares of AWK were purchased for $40000


We can see Archer's current stock holdings and capital with the <b>shares</b> and <b>capital</b> variables:

In [14]:
Archer.shares

{'GOLD': 1383.6, 'AWK': 277.24}

In [15]:
Archer.capital

20000

Additionally, you can see a history of Archer's transactions with the function <b>getLedger</b>, which returns a list of transactions in the form of a Pandas DataFrame:

In [16]:
Archer.getLedger()

,date,action,symbol,Δshares,Δdollars,capital
0,2020-07-29 18:00:00,buy,GOLD,1383.60,-40000,60000
1,2020-07-24 15:00:00,buy,AWK,277.24,-40000,20000


On thing you might notice is that the dates are different in the ledger above, even though we bought the stock at the same time. This is because when we called the <b>buy</b> function, <u>we did not specify a date</u>. If no date is specified, the stock is purchased at the open price of the first point in the dataset. Our two stocks have a different first entry in their datasets, so the dates are different in the ledger.

## Selling shares

Now, Mr. Archer is getting a little impatient and would like his shares liquidated so that we can go buy a bunch of sand to rub in Woodhouse's eyes. What kind of sand, you ask? Coarse.]]

We can get rid of Sterling's stock with the <b>sell</b> function. In the sell function, you can specify the stock you want to sell, the amount in shares or dollars, and the index/date/time in which to sell. If you don't put an amount all holdings are sold and if you don't put in an index the last one is used.

In [17]:
Archer.sell('GOLD')

No value entered, all shares will be sold
1383.6 shares of GOLD were sold for $39695.48


In [18]:
Archer.sell('AWK')

No value entered, all shares will be sold
277.24 shares of AWK were sold for $40770.91


In [19]:
Archer.capital

100466.39000000001

In [20]:
Archer.getLedger()

,date,action,symbol,Δshares,Δdollars,capital
0,2020-07-29 18:00:00,buy,GOLD,1383.60,-40000.00,60000.00
1,2020-07-24 15:00:00,buy,AWK,277.24,-40000.00,20000.00
2,2020-08-03 19:30:00,sell,GOLD,-1383.60,39695.48,59695.48
3,2020-08-03 16:30:00,sell,AWK,-277.24,40770.91,100466.39


So, we can see that our superagent spy actually made money! What a surprise...

## Come back next time for instructions on how to use this framework to implement an algorithm!